In [18]:
import numpy as np
from hmmlearn import hmm_c
from hmmlearn import hmm
import alfabeto_data.dissertation_images as di
from alfabeto_sources.all_sources import *
from alfabeto_code.AlfabetoConverter import transposed_pc_chords_noMMD
from Continuo.ContinuoConverter import figure_intervals_pc
import csv
import copy
from scipy.spatial.distance import euclidean, cdist
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, to_agraph
from sklearn.externals import joblib

In [6]:
# X = di.python_hmm_corpus(di.monteverdi_continuo, di.modal_major)
X = di.python_hmm(di.GetComposer.all_kaps, di.modal_major, 'chords')

In [7]:
n_samples = 500
C = np.array([[0., -0.1], [1.7, .4]])
XX = np.r_[np.dot(np.random.randn(n_samples, 2), C),
          .7 * np.random.randn(n_samples, 2) + np.array([-6, 3])]

[{1: {'alfabeto': 'E  D X  |C A  C |+  R +  |CACA C |A||EFRF| EFBEF|IC A  + BA|CA',
   'continuo': '22299eee|6677722|444ee444|276ee22|7||08e4|424024|920e02420e|27',
   'data': {'final': 7, 'key': 'n'}},
  2: {'alfabeto': 'O| EG|GHGL|AB|BGHGHL|CA',
   'continuo': '7|725|5t50|70|0524t3|27',
   'data': {'final': 7, 'key': 'f'}},
  3: {'alfabeto': 'H  LE|   E  |G H  |G    |A  B  M  |M  C  A||A|C O|G E  |H OI C |M     |H M |DC O C|   A',
   'continuo': 'ttt02|222444|55ttt|55554|45700e023|20t2227||7|227|54220|t979922|032320|tt32|0t9753|2027',
   'data': {'final': 7, 'key': 'f'}},
  4: {'alfabeto': 'BBA|A BD|G A |BA|A D |F  B C |AH    |HBG |ABH|H  I|CGFGAB',
   'continuo': '007|7709|5577|07|7799|eee0022|799999|9055|70t|t0t9|754570',
   'data': {'final': 0, 'key': 'n'}},
  5: {'alfabeto': 'B   A |D B G A |B   D R |F   D F |I C I CFI ',
   'continuo': '000075|44024577|00000eee|44449944|9997524499',
   'data': {'final': 9, 'key': 'n'}}},
 {1: {'alfabeto': 'A DC|ABGBGA|B |R|  F|IC|OCODCA|B CA|| R

In [ ]:
from hmmlearn import base_c

In [ ]:
X
NUMERALS = copy.deepcopy(X[-1])

In [15]:
Z_c = hmm_c.MultinomialHMM(n_components=3, verbose=False,n_iter=1000,tol=.01)
Z = hmm.MultinomialHMM(n_components=3, verbose=False,n_iter=1000,tol=.01)
Z.n_features = len(X[2])
# FIT_DATA_c = Z_c.fit(X[0], X[1])
# FIT_DATA = Z.fit(X[0], X[1])

In [17]:
import timeit
def cymaker():
    return Z_c.fit(X[0], X[1])
def pymaker():
    return Z.fit(X[0], X[1])
cy = timeit.timeit('''cymaker()''')
py = timeit.timeit('''pymaker()''')

print(cy, py)
print('Cython is {}x faster'.format(py/cy))

NameError: name 'cymaker' is not defined

In [ ]:
FIT_DATA.score(X[0], X[1])

In [ ]:
FIT_DATA.score(X[0], X[1])

In [ ]:
N = {x:NUMERALS.index(x) for x in NUMERALS}
I = [N['I']]
ii = [N['ii']]
iii = [N['iii']]
IV = [N['IV']]
V = [N['V']]
bVII = [N['bVII']]

In [ ]:
FIT_DATA.predict_proba([[NUMERALS.index('I')], [NUMERALS.index('IV')], [NUMERALS.index('V')]])
e = FIT_DATA.score_samples([I, V, I, IV, I, IV, V, I])

import math
print(e[0])
print(e[1])
# math.exp(e)

In [ ]:
tp = 1
YY = hmm.MultinomialHMM(n_components=3, verbose=True, transmat_prior = tp, n_iter=1000,tol=.1)
ff = YY.fit(X[0], X[1])


In [ ]:
YY.fit(X[0], X[1])

In [ ]:
FIT_DATA = Y.fit(X[0], X[1])

In [ ]:
emit = FIT_DATA.emissionprob_
X[2]

In [ ]:
ESTTR11 = FIT_DATA.transmat_
TOKEN_NAMES=X[2]
ESTEMIT11 = FIT_DATA.emissionprob_
K=3
pickleit = {'fit': FIT_DATA, 'tokens': TOKEN_NAMES}
# joblib.dump(pickleit, '/home/daniel/Desktop/hmmpickle/alfmajorchords6.pkl')

In [ ]:
jj = joblib.load('/home/daniel/Desktop/hmm/alfabeto_continuo/minor/3/3.pkl')

In [ ]:
jj['fit'].transmat_

In [ ]:
emission_prob = []
for x in ESTEMIT11:
    hey = []
    for y in x:
        if y >= .01:
            hey.append(y)
        else:
            hey.append(0)
    emission_prob.append(hey)

In [ ]:
def csv_function(f_variable, path):
    with open(path, 'w', newline='') as fp:
        a = csv.writer(fp, delimiter=',')
        a.writerows(f_variable)

In [ ]:
csv_function(emission_prob, '/home/daniel/Desktop/emit_moz_maj.csv')

In [ ]:
#compute marginal distribution
trn = copy.deepcopy(ESTTR11).conj().transpose()
(D,V) = copy.deepcopy(np.linalg.eig(trn))
item=max(D)
idx = 0
for x, y in zip(D, range(len(D))):
    if x == item:
        idx = y
f_marginal=abs(V[:,idx])/sum(abs(V[:,idx]))
#whew!

#compute P(F|C)
k = []
for x in ESTEMIT11:
    k.append(np.zeros(np.size(x)))
PFC = np.array(k)
for jj in range(1,len(ESTEMIT11[0])+1):
    for ii in range(1,len(ESTEMIT11[:,0])+1):
        PFC[ii-1][jj-1]=ESTEMIT11[ii-1][jj-1]*f_marginal[ii-1]/jj
    PFC[:,jj-1]=PFC[:,jj-1]/sum(PFC[:,jj-1])
    b = PFC[:,jj-1]
    a = sum(PFC[:,jj-1])

In [ ]:
#Most Likely P(F|C)
[midx,_] = PFC.max(0),PFC.argmax(0)
state_dict = {}
for kk in range(K):
    state_dict[kk] = {}
    for ll in range(len(midx)):
        if PFC[kk][ll]>0.01:
            state_dict[kk][TOKEN_NAMES[ll]]=PFC[kk][ll]


In [ ]:
CSV_LIST = [TOKEN_NAMES]
for state in range(len(state_dict)):
    token_pct = []
    for t in TOKEN_NAMES:
        num = 0
        for a,b in state_dict[state].items():
            if t == a:
                token_pct.append(b)
                num+=1
        if num == 0:
            token_pct.append(0)
    CSV_LIST.append(token_pct)

In [ ]:
csv_function(CSV_LIST, '/home/daniel/Desktop/alf_major_functions.csv')

In [ ]:
emission_prob = copy.deepcopy(ESTEMIT11)
new_csv = copy.deepcopy(np.array(CSV_LIST[1:]))
# print(TOKEN_NAMES)
# print(emission_prob.shape)
token_reduced = []
delete = []
for y in range(len(emission_prob[0])):
    if sum(emission_prob[0:,y]) <= .03:
        delete.append(y)
    else:
        token_reduced.append(TOKEN_NAMES[y])
emission_reduced = np.delete(new_csv, delete, axis=1)
# print(emission_reduced.shape)
# print(len(token_reduced))
# print(token_reduced)
# print(emission_reduced)

In [ ]:
reduced_csv = [token_reduced]
for x in emission_reduced:
    reduced_csv.append(x)
csv_function(reduced_csv, '/home/daniel/Desktop/reduced_pal_major4.csv')

In [ ]:
# emission = np.swapaxes(np.array(CSV_LIST[1:]), 0, 1)
# # emission = np.array(emission_reduced)
# neato_data = cdist(emission, emission, 'euclidean')*10
# # neato_data = []
# # for x in emission:
# #     temp = []
# #     for y in emission:
# #         temp.append(euclidean(x, y))
# #     neato_data.append(temp)
# # print(neato_data)
# print(len(token_reduced))

# dt = [('len', float)]

# A = np.array(neato_data).view(dt)
# G = nx.from_numpy_matrix(A)
# relabeled_nodes = {}
# for x, y in zip(range(len(neato_data[0])), TOKEN_NAMES):
#     relabeled_nodes[x] = y
#     # print(y[1:-1])
# G = nx.relabel_nodes(G, relabeled_nodes)

# G = to_agraph(G)

# G.node_attr.update(style='filled', fontsize=20)
# G.edge_attr.update(color='black', style='invis')
# print(G)
# #  ♭ ♮ ...for copy and paste
# G.draw('/home/daniel/Desktop/tryit.pdf', format='pdf', prog='neato')

In [ ]:
ESTTR11
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, to_agraph
import pygraphviz as pgv
labels = ['S', 'T', 'D']
G = nx.DiGraph()
for x, y in zip(ESTTR11, labels):
    for a, b in zip(x, labels):
        G.add_edge(y, b, penwidth=a*5)
edge_weights = []
edges = G.edges(data=True)
print('edges:', edges)
for x in edges:
    edge_weights.append(x[2]['penwidth'])
nodes = G.nodes()
print('nodes:', nodes)
for x in nodes:
    G.node[x]['shape'] = 'circle'
G = to_agraph(G)
# G=pgv.AGraph(ranksep=10)
G.draw('/home/daniel/Desktop/function.png', format='png', prog='dot')

In [ ]:
pp = FIT_DATA.predict_proba(at)
bigF = []
for x in range(len(pp)):
    counter = 0
    func = 0
    for y in pp[x]:
        if y >= .9:
            counter += 1
            func += func_guess[x]
    if counter > 0:
        bigF.append(func)
    else:
        bigF.append('-')

In [ ]:
a

In [ ]:
felici_numerals = transposed_pc_chords_noMMD(di.GetComposer.all_kaps[0][1]['alfabeto'], 'all')
felici_numb = [[TOKEN_NAMES.index(i)] for i in felici_numerals]
# f_nums = di.python_hmm(felici, di.modal_major)
f_guess = FIT_DATA.predict(felici_numb)
pp = FIT_DATA.predict_proba(felici_numb)
Labels = ['T', 'D', 'S']

In [ ]:
bigF = []
reduced_numerals = []
bigF2 = []
for x in range(len(pp)):
    counter = 0
    func = 0
    for y in pp[x]:
        if y >= .95:
            counter += 1
            func += f_guess[x]
            reduced_numerals.append(felici_numb[x])
    if counter > 0:
        bigF.append(Labels[func])
    else:
        bigF.append('-')
func_surface = [Labels[n] for n in f_guess]


In [ ]:
csv_felici = [felici_numerals, f_guess, [f[0] for f in pp], [f[1] for f in pp], [f[2] for f in pp]]

In [ ]:
csv_function(csv_felici, '/home/daniel/Desktop/hierarchy.csv')

In [ ]:
def hierarchy_maker(corpus_function):
    f_numbers = [[TOKEN_NAMES.index(i)] for i in corpus_function]
    reduced_numerals = copy.deepcopy(f_numbers)
    rereduced_numerals = []
    print([TOKEN_NAMES[x[0]] for x in reduced_numerals])
    
    while reduced_numerals != rereduced_numerals:
        if len(rereduced_numerals) > 0:
            reduced_numerals = rereduced_numerals
        f_guess = FIT_DATA.predict(reduced_numerals)
        pp = FIT_DATA.predict_proba(reduced_numerals)
        print(len(pp))
        rereduced_numerals = []
        for x in range(len(pp)):
            counter = 0
            func = 0
            for y in pp[x]:
                if y >= .99:
                    counter += 1
                    func += f_guess[x]
                    rereduced_numerals.append(reduced_numerals[x])
#         rereduced_numerals = []
        print([TOKEN_NAMES[x[0]] for x in reduced_numerals])
        print(len(reduced_numerals))
    print()
#         if counter > 0:
#             bigF.append(Labels[func])
#         else:
#             bigF.append('-')
#     func_surface = [Labels[n] for n in f_guess]

In [ ]:
hierarchy_maker(transposed_pc_chords_noMMD(di.GetComposer.all_kaps[0][1]['alfabeto'], 'all'))

In [ ]:
hierarchy_maker

In [ ]:
np.insert(emit, 0, 666, axis=1)